# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [1]:
%pip install -qU datasets pinecone-client==3.1.0 sentence-transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [2]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True,
    trust_remote_code=True
).shuffle(seed=960)

wikipedia_snippets_streamed.py:   0%|          | 0.00/4.58k [00:00<?, ?B/s]

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [3]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [4]:
# filter only documents with History as section_title - Replace None with your code
history = wiki_data.filter(lambda example: example['section_title'] == 'History')

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [5]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

counter = 0
docs = []

# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # extract the fields we need - article, section, and passage
    doc = {
        'article': d['article_title'],
        'section': d['section_title'],
        'passage': d['passage_text']
    }
    docs.append(doc)
    # increase the counter on every iteration
    counter += 1
    # Break the loop when we reach the desired number of documents
    if counter >= total_doc_count:
        break

  0%|          | 0/50000 [00:00<?, ?it/s]

In [6]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article,section,passage
0,Taupo District,History,was not until the 1950s that the region starte...
1,The Bishop Wand Church of England School,History,The Bishop Wand Church of England School Histo...
2,Surface Hill Uniting Church,History,in perpetual reminder that work and worship go...
3,The Electras (band),History,"as its B-side. However, copies of the single, ..."
4,Swanton House,History,it. Lane provided funds for restoration by the...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [7]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [8]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [9]:
index_name = "wikipedia_history_index" #give your index a meaningful name

In [10]:
import time
import json
import os
from pathlib import Path

# Define a class to handle document storage and retrieval
class SimpleDocumentStore:
    def __init__(self, index_name):
        self.index_name = index_name
        self.documents = {}
        self.document_count = 0
        self.index_file = f"{index_name}.json"

        # Check if index already exists
        if os.path.exists(self.index_file):
            # If it exists, you might want to delete and recreate it
            os.remove(self.index_file)
            print(f"Index '{index_name}' already existed. Deleted and recreated.")
        else:
            print(f"Index '{index_name}' created successfully.")

        # Create empty index file
        with open(self.index_file, 'w') as f:
            json.dump({}, f)

    def add_document(self, doc_id, document):
        self.documents[doc_id] = document
        self.document_count += 1

    def search(self, query, top_k=10):
        # Simple search implementation - could be enhanced with better text matching
        results = []
        for doc_id, doc in self.documents.items():
            # Check if query appears in the passage
            if query.lower() in doc['passage'].lower():
                results.append((doc_id, doc))

        # Sort by simple relevance (could be improved)
        results = sorted(results, key=lambda x: x[1]['passage'].lower().count(query.lower()), reverse=True)
        return results[:top_k]

    def save(self):
        with open(self.index_file, 'w') as f:
            json.dump(self.documents, f)

    def stats(self):
        return {
            "doc_count": self.document_count
        }

# Initialize our document store
doc_store = SimpleDocumentStore(index_name)

# Verify that the index is empty
stats = doc_store.stats()
doc_count = stats["doc_count"]
print(f"Document count in index: {doc_count}")

Index 'wikipedia_history_index' created successfully.
Document count in index: 0


# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [11]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base', device=device)
retriever

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [12]:
import numpy as np
import pickle
import os
from tqdm.auto import tqdm

# Create a simple vector database class
class SimpleVectorDB:
    def __init__(self, name):
        self.name = name
        self.embeddings = {}  # id -> embedding vector
        self.metadatas = {}   # id -> metadata dict
        self.documents = {}   # id -> document text
        self.file_path = f"{name}_vector_db.pkl"

        # Load if exists
        if os.path.exists(self.file_path):
            self.load()

    def add(self, ids, embeddings, metadatas, documents):
        for i, doc_id in enumerate(ids):
            self.embeddings[doc_id] = embeddings[i]
            self.metadatas[doc_id] = metadatas[i]
            self.documents[doc_id] = documents[i]

    def search(self, query_embedding, top_k=5):
        # Convert query embedding to numpy for calculations
        query_embedding = np.array(query_embedding)

        # Calculate cosine similarity
        results = []
        for doc_id, embedding in self.embeddings.items():
            embedding = np.array(embedding)
            # Cosine similarity
            similarity = np.dot(query_embedding, embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(embedding))
            results.append((doc_id, similarity))

        # Sort by similarity (highest first)
        results.sort(key=lambda x: x[1], reverse=True)

        # Return top k results with their metadata and documents
        top_results = []
        for doc_id, score in results[:top_k]:
            top_results.append({
                'id': doc_id,
                'score': float(score),
                'metadata': self.metadatas[doc_id],
                'document': self.documents[doc_id]
            })

        return top_results

    def count(self):
        return len(self.embeddings)

    def save(self):
        with open(self.file_path, 'wb') as f:
            pickle.dump({
                'embeddings': self.embeddings,
                'metadatas': self.metadatas,
                'documents': self.documents
            }, f)
        print(f"Vector database saved to {self.file_path}")

    def load(self):
        with open(self.file_path, 'rb') as f:
            data = pickle.load(f)
            self.embeddings = data['embeddings']
            self.metadatas = data['metadatas']
            self.documents = data['documents']
        print(f"Vector database loaded from {self.file_path}")

# Initialize our simple vector database
index = SimpleVectorDB(index_name)

# we will use batches of 64
batch_size = 64

# Create embeddings for the passage_text variable and include the metadata in each batch
for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(df))

    # extract batch
    batch = df.iloc[i:i_end]

    # generate embeddings for batch
    embeddings = retriever.encode(batch['passage'].tolist(), convert_to_tensor=True)

    # add embeddings to index with metadata
    ids = []
    embs = []
    metas = []
    docs = []

    for j, (idx, row) in enumerate(batch.iterrows()):
        ids.append(str(idx))
        embs.append(embeddings[j].cpu().numpy().tolist())
        metas.append({
            'article': row['article'],
            'section': row['section'],
            'passage': row['passage']
        })
        docs.append(row['passage'])

    index.add(ids=ids, embeddings=embs, metadatas=metas, documents=docs)

# Save the vector database to disk
index.save()

# check that we have all vectors in index
print(f"Total documents indexed: {index.count()}")

  0%|          | 0/782 [00:00<?, ?it/s]

Vector database saved to wikipedia_history_index_vector_db.pkl
Total documents indexed: 50000


# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [13]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [20]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode(query, convert_to_tensor=True).cpu().numpy().tolist()
    # search pinecone index for context passage with the answer
    xc = index.search(xq, top_k=top_k)
    return xc

In [38]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    # context is a list of dictionaries, and we need to access the 'metadata' dictionary within each element
    # Modified to handle the custom 'index' search result format
    context = [f"<P> {m['metadata']['passage']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [39]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

[{'id': '2727',
  'score': 0.6911800184836769,
  'metadata': {'article': 'Electric power system',
   'section': 'History',
   'passage': "Electric power system History In 1881, two electricians built the world's first power system at Godalming in England. It was powered by two waterwheels and produced an alternating current that in turn supplied seven Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. However, supply to the lamps was intermittent and in 1882 Thomas Edison and his company, The Edison Electric Light Company, developed the first steam-powered electric power station on Pearl Street in New York City. The Pearl Street Station initially powered around 3,000 lamps for 59 customers. The power station generated direct current and"},
  'document': "Electric power system History In 1881, two electricians built the world's first power system at Godalming in England. It was powered by two waterwheels and produced an alternating current that in turn supplied seven 

In [40]:
from pprint import pprint

In [42]:
# format the query in the form generator expects the input
query = format_query(query, result)
pprint(query)

('question: when was the first electric power system built? context: <P> '
 "Electric power system History In 1881, two electricians built the world's "
 'first power system at Godalming in England. It was powered by two '
 'waterwheels and produced an alternating current that in turn supplied seven '
 'Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. '
 'However, supply to the lamps was intermittent and in 1882 Thomas Edison and '
 'his company, The Edison Electric Light Company, developed the first '
 'steam-powered electric power station on Pearl Street in New York City. The '
 'Pearl Street Station initially powered around 3,000 lamps for 59 customers. '
 'The power station generated direct current and')


The output looks great. Now let's write a function to generate answers.

In [43]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [44]:
generate_answer(query)

('The first electric power system was built in 1881 at Godalming in England. '
 'It was powered by two waterwheels and produced alternating current that in '
 'turn supplied seven Siemens arc lamps')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [45]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context)
generate_answer(query)

('The first wireless message was sent in 1866 by Mahlon Loomis. He used a kite '
 'to send a signal through the atmosphere. The kite was attached to a cable '
 'that')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [47]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

TypeError: list indices must be integers or slices, not str

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [ ]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

In [ ]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

Let’s finish with a final few questions.

In [48]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context)
generate_answer(query)

('The War of Currents was a series of events in the early 1900s between Edison '
 'and Westinghouse. Edison was the inventor of alternating current, and '
 'Westinghouse was the inventor of')


In [ ]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

In [ ]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

As we can see, the model can generate some decent answers.

#### Add a few more questions